In [15]:
import yfinance as yf
import pandas as pd 

In [16]:
df=yf.download("RELIANCE.NS", start="2023-01-01", end="2025-11-30",auto_adjust=True)
df.columns = df.columns.droplevel(level=1)
df

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Date,,,,,
2023-01-02,1175.889648,1177.304802,1163.244669,1164.066352,5316175
2023-01-03,1167.284546,1174.565705,1163.062026,1170.936536,7658932
2023-01-04,1149.709595,1169.110728,1147.632506,1167.261851,9264891
2023-01-05,1147.655396,1157.858149,1143.067616,1151.969321,13637099
2023-01-06,1158.086182,1163.130505,1149.595379,1153.407118,6349597
...,...,...,...,...,...
2025-11-24,1535.900024,1550.000000,1531.800049,1550.000000,18433951
2025-11-25,1539.699951,1559.599976,1525.099976,1535.900024,15033482
2025-11-26,1569.900024,1571.599976,1540.500000,1542.300049,14054299


In [17]:
#sma , ema , macd , rsi , bollinger bands, 


SIMPLE MOVING AVERAGE

In [18]:
df["SMA"]= df["Close"].rolling(window=20).mean()
df

Price,Close,High,Low,Open,Volume,SMA
Date,,,,,,
2023-01-02,1175.889648,1177.304802,1163.244669,1164.066352,5316175,NaN
2023-01-03,1167.284546,1174.565705,1163.062026,1170.936536,7658932,NaN
2023-01-04,1149.709595,1169.110728,1147.632506,1167.261851,9264891,NaN
2023-01-05,1147.655396,1157.858149,1143.067616,1151.969321,13637099,NaN
2023-01-06,1158.086182,1163.130505,1149.595379,1153.407118,6349597,NaN
...,...,...,...,...,...,...
2025-11-24,1535.900024,1550.000000,1531.800049,1550.000000,18433951,1504.160004
2025-11-25,1539.699951,1559.599976,1525.099976,1535.900024,15033482,1506.800000
2025-11-26,1569.900024,1571.599976,1540.500000,1542.300049,14054299,1510.085004


EXPONENTIAL MOVING AVERAGE

In [19]:
df["EMA"]= df["Close"].ewm(span=20, adjust=False).mean()

MOVING AVERAGE CONVERGENCE DIVERGENCE 


In [20]:
def macd_crossover(df):
    df["EMA_12"] = df["Close"].ewm(span=12, adjust=False).mean()
    df["EMA_26"] = df["Close"].ewm(span=26, adjust=False).mean()    
    df["MACD"] = df["EMA_12"] - df["EMA_26"]
    df["Signal_Line"] = df["MACD"].ewm(span=9, adjust=False).mean()
    
    signals = [0]  
    for i in range(1, len(df)):
        previous_macd = df["MACD"].iloc[i-1]
        current_macd = df["MACD"].iloc[i]
        previous_signal = df["Signal_Line"].iloc[i-1]    
        current_signal = df["Signal_Line"].iloc[i]
        if previous_macd < previous_signal and current_macd > current_signal:
            signals.append(1)  
        elif previous_macd > previous_signal and current_macd < current_signal:
            signals.append(-1)  
        else:
            signals.append(0)  
    df["Signal"] = signals
    return df
df=macd_crossover(df)




RSI

In [ ]:
def calculate_rsi(df, period=14):
    change = df["Close"]-df["Close"].shift(1)
    gain = (change.where(change > 0, 0)).rolling(window=period).mean()
    loss = (-change.where(change < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    df["RSI"] = rsi
    return df
df=calculate_rsi(df)

STOCHASTIC OSCILLATOR

In [ ]:
def stochastic_oscillator(df, kperiod=14, dperiod=3):
    low = df["Low"].rolling(window=kperiod).min()
    high = df["High"].rolling(window=kperiod).max()
    df["%K"] = 100 * ((df["Close"] - low) / (high - low))
    df["%D"] = df["%K"].rolling(window=dperiod).mean()
    return df

BOLLINGER BANDS


In [ ]:
def bollinger_bands(df, window=20):
    df["mainband"] = df["Close"].rolling(window=window).mean()
    df["Stddev"] = df["Close"].rolling(window=window).std()
    df["Upperband"] = df["mainband"] + (2*df["Stddev"])
    df["Lowerband"] = df["mainband"] - (2*df["Stddev"])
    return df